In [1]:
!pip install pydantic pydantic-ai python-dotenv ipython devtools

In [2]:
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
openai_api_key = os.environ.get("OPENAI_API_KEY", "")
google_api_key = os.environ.get("GEMINI_API_KEY", "")

# Set API keys only if they're not already set
if not os.environ.get("OPENAI_API_KEY") and openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if not os.environ.get("GEMINI_API_KEY") and google_api_key:
    os.environ["GEMINI_API_KEY"] = google_api_key

# Add error handling for missing API keys
# Check which API keys are missing and provide specific warnings
missing_keys = []
if not os.environ.get("OPENAI_API_KEY"):
    missing_keys.append("OPENAI_API_KEY")
if not os.environ.get("GEMINI_API_KEY"):
    missing_keys.append("GEMINI_API_KEY")

if missing_keys:
    print(f"Warning: The following API key(s) are missing: {', '.join(missing_keys)}")
    print(f"Some functionality using {', '.join(missing_keys)} may be limited.")
# Display a message indicating that the API keys have been loaded
display(Markdown("### API keys have been loaded successfully."))

USE_GEMINI= True



Some functionality using OPENAI_API_KEY may be limited.


### API keys have been loaded successfully.

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from pydantic import BaseModel, Field
from typing import Optional
from devtools import debug
from dataclasses import dataclass


class YearlyValue(BaseModel):
    """Model for values that can have current and previous year figures"""
    CFY: float
    PFY: Optional[float] = None


class StatementOfProfitOrLoss(BaseModel):
    """Financial statement model for profit or loss statement with current and previous year values"""
    
    # Fields with financial definitions and common terms
    revenue: YearlyValue = Field(..., description="The income generated from normal business operations. Common terms: Revenue, Turnover, Sales, Revenue from contracts with customers")
    
    other_income: Optional[YearlyValue] = Field(None, description="Income from activities unrelated to main business operations, such as interest, dividends, or rental income. Common terms: Other income, Other operating income, Other revenue")
    
    employee_benefits_expense: Optional[YearlyValue] = Field(None, description="All costs related to employee compensation, including salaries, wages, bonuses, CPF contributions, and employee benefits. Common terms: Employee benefits expense, Staff costs, Personnel expenses, Employee costs, Salaries and wages")
    
    depreciation_expense: Optional[YearlyValue] = Field(None, description="The systematic allocation of tangible asset costs over their useful lives. Common terms: Depreciation expense, Depreciation, Depreciation of property, plant and equipment")
    
    amortisation_expense: Optional[YearlyValue] = Field(None, description="The systematic allocation of intangible asset costs over their useful lives. Common terms: Amortisation expense, Amortization expense, Amortisation, Amortization, Amortisation of intangible assets")
    
    repairs_maintenance_expense: Optional[YearlyValue] = Field(None, description="Costs incurred to maintain or repair property, equipment, and other assets. Common terms: Repairs and maintenance, Repairs and maintenance expense, Maintenance costs, Repairs expense")
    
    sales_marketing_expense: Optional[YearlyValue] = Field(None, description="Expenses related to promoting and selling products/services, including advertising, marketing campaigns, and sales staff costs. Common terms: Sales and marketing expense, Marketing expenses, Selling expenses, Marketing and distribution expenses")
    
    other_expenses: Optional[YearlyValue] = Field(None, description="Operating expenses not classified in other specific categories, often including administrative, utility, and general business costs. Common terms: Other expenses, Other operating expenses, General expenses, Administrative expenses")
    
    other_gains_losses: Optional[YearlyValue] = Field(None, description="Gains or losses from non-operating activities or one-time events, including foreign exchange differences or asset disposals. Common terms: Other gains and losses, Foreign exchange gain/(loss), Fair value changes, Forex gain/loss")
    
    finance_costs_net: Optional[YearlyValue] = Field(None, description="The net cost of financing, typically interest expense minus interest income, including loan interest, bond interest, and other financing costs. Common terms: Finance costs, Finance income, Finance expense, Interest expense, Interest income, Net finance costs")
    
    share_of_profit_loss_associates: Optional[YearlyValue] = Field(None, description="The entity's share of profits or losses from investments in associates and joint ventures accounted for using the equity method. Common terms: Share of profit of associates, Share of results of associates, Share of profit (loss) of associates and joint ventures")
    
    profit_loss_before_taxation: YearlyValue = Field(..., description="The profit or loss before income tax expense is considered, representing operational performance before tax obligations. Common terms: Profit before tax, Profit before taxation, Earnings before tax, Profit before income tax")
    
    income_tax_expense_benefit: YearlyValue = Field(..., description="The amount of income taxes payable or recoverable for the period, including current and deferred tax. Common terms: Income tax expense, Tax expense, Taxation, Income tax benefit, Tax benefit")
    
    profit_loss_discontinued_operations: Optional[YearlyValue] = Field(None, description="The profit or loss from business components that have been disposed of or classified as held for sale, shown net of tax. Common terms: Profit from discontinued operations, Loss from discontinued operations, Discontinued operations")
    
    total_profit_loss: Optional[YearlyValue] = Field(None, description="The bottom-line profit or loss for the financial period after all expenses, including tax. Common terms: Profit for the year, Loss for the year, Net profit, Net loss, Profit for the period, Total comprehensive income")
    
    profit_loss_attributable_to_owners: Optional[YearlyValue] = Field(None, description="The portion of total profit or loss that belongs to the shareholders or owners of the parent company. Common terms: Profit attributable to owners, Profit attributable to shareholders, Profit attributable to equity holders")
    
    profit_loss_attributable_to_non_controlling: Optional[YearlyValue] = Field(None, description="The portion of total profit or loss that belongs to minority shareholders who own less than 50% of a subsidiary. Common terms: Profit attributable to non-controlling interests, Profit attributable to minority interests")


In [5]:
# Define the system prompt separately for better readability and maintenance
FINANCIAL_STATEMENT_PROMPT = """You are a Singapore financial reporting specialist who converts annual reports to XBRL format.
Your task is to extract and map financial data from input reports into standardized Statement of Profit or Loss fields.

IMPORTANT MAPPING GUIDELINES:
1. For each financial item, extract both Current Financial Year (CFY) and Previous Financial Year (PFY) values when available.
   - CFY values are required for mandatory fields
   - PFY values should be included when available, but can be omitted if not present

2. Pay close attention to financial terminology and map fields based on accounting meaning rather than exact names.
3. Map fields according to Singapore XBRL taxonomy and reporting standards.

4. Match input fields to the StatementOfProfitOrLoss model structure, handling common variations such as:
   - 'revenue' matches with: 'Revenue', 'Turnover', 'Sales', etc.
   - 'employee_benefits_expense' matches with: 'Employee Benefits Expense', 'Staff costs', 'Personnel expenses', etc.
   - 'depreciation_expense' matches with: 'Depreciation', 'Depreciation of Property, Plant and Equipment', etc.
   - 'amortisation_expense' matches with: 'Amortisation', 'Amortization', 'Amortisation of Intangible Assets', etc.
   - 'repairs_maintenance_expense' matches with: 'Repairs and Maintenance', 'Maintenance costs', etc.
   - 'sales_marketing_expense' matches with: 'Sales and Marketing', 'Marketing expenses', 'Advertising costs', etc.
   - 'other_expenses' matches with: 'Other Operating Expenses', 'General expenses', 'Administrative expenses', etc.
   - 'other_gains_losses' matches with: 'Foreign Exchange Gain/(Loss)', 'Other gains', 'Fair value changes', etc.
   - 'finance_costs_net' matches with: 'Finance Costs', 'Interest expense', 'Financial expenses', etc.
   - 'profit_loss_before_taxation' matches with: 'Profit Before Tax', 'Profit before income tax', etc.
   - 'income_tax_expense_benefit' matches with: 'Income Tax Expense', 'Tax expense', 'Taxation', etc.
   - 'total_profit_loss' matches with: 'Profit For The Year', 'Net profit', 'Total comprehensive income', etc.

5. The input data may present CFY/PFY values in different formats:
   - As separate fields: "Revenue 2024" and "Revenue 2023"
   - As nested objects: {"Revenue": {"CFY": 1000000, "PFY": 900000}}
   - In a tabular format where values are in different columns
   - Sometimes labeled as "Current Year" and "Previous Year"

6. Focus on extracting numerical values only - ignore metadata fields like company names, dates, or notes.

7. If needed information isn't explicitly given but can be calculated from other fields, perform the calculation.
   
8. Return all detected fields that correspond to the Statement of Profit or Loss structure with each value containing:
   - CFY: The current financial year value (required for mandatory fields)
   - PFY: The previous financial year value (if available)

9. The three required fields that must have at least CFY values are:
   - revenue
   - profit_loss_before_taxation
   - income_tax_expense_benefit
"""

In [6]:
dummy_data = {
    "revenue": 1000000.0,
    "other_income": 50000.0,
    "staff_costs": 350000.0,  # should map to employee_benefits_expense
    "depreciation": 75000.0,  # should map to depreciation_expense
    "amortization": 25000.0,  # should map to amortisation_expense
    "repairs_and_maintenance": 30000.0,  # should map to repairs_maintenance_expense
    "marketing_expenses": 120000.0,  # should map to sales_marketing_expense
    "general_expenses": 80000.0,  # should map to other_expenses
    "foreign_exchange_gain": 15000.0,  # should map to other_gains_losses
    "interest_expense": 40000.0,  # should map to finance_costs_net
    "share_of_results_from_associates": 35000.0,  # should map to share_of_profit_loss_associates
    "profit_before_tax": 380000.0,  # should map to profit_loss_before_taxation
    "tax_expense": 65000.0,  # should map to income_tax_expense_benefit
    "loss_from_discontinued_ops": 10000.0,  # should map to profit_loss_discontinued_operations
    "net_profit": 305000.0,  # should map to total_profit_loss
    "profit_attributable_to_shareholders": 290000.0,  # should map to profit_loss_attributable_to_owners
    "profit_attributable_to_minority_interests": 15000.0,  # should map to profit_loss_attributable_to_non_controlling
    
    # Some noise fields
    "company_name": "Test Company Ltd",
    "year": 2024,
    "current_ratio": 1.5
}

In [7]:
dummy_profit_loss_data = {
    "Revenue": {
        "CFY": 12500000,
        "PFY": 11250000
    },
    "Other Income": {
        "CFY": 350000,
        "PFY": 275000
    },
    "Employee Benefits Expense": {
        "CFY": -4750000,
        "PFY": -4250000
    },
    "Depreciation of Property, Plant and Equipment": {
        "CFY": -780000,
        "PFY": -720000
    },
    "Amortisation of Intangible Assets": {
        "CFY": -180000,
        "PFY": -165000
    },
    "Repairs and Maintenance": {
        "CFY": -320000,
        "PFY": -280000
    },
    "Sales and Marketing": {
        "CFY": -1450000,
        "PFY": -1350000
    },
    "Other Operating Expenses": {
        "CFY": -970000,
        "PFY": -850000
    },
    "Foreign Exchange Gain/(Loss)": {
        "CFY": 85000,
        "PFY": -120000
    },
    "Finance Costs": {
        "CFY": -325000,
        "PFY": -290000
    },
    "Share of Results of Associates": {
        "CFY": 620000,
        "PFY": 580000
    },
    "Profit Before Tax": {
        "CFY": 4780000,
        "PFY": 4080000
    },
    "Income Tax Expense": {
        "CFY": -860000,
        "PFY": -735000
    },
    "Profit From Continuing Operations": {
        "CFY": 3920000,
        "PFY": 3345000
    },
    "Loss From Discontinued Operations": {
        "CFY": -150000,
        "PFY": -275000
    },
    "Profit For The Year": {
        "CFY": 3770000,
        "PFY": 3070000
    },
    "Attributable to:": {
        "Owners of the Company": {
            "CFY": 3620000,
            "PFY": 2950000
        },
        "Non-controlling Interests": {
            "CFY": 150000,
            "PFY": 120000
        }
    },
    "Earnings Per Share (in cents)": {
        "Basic": {
            "CFY": 18.5,
            "PFY": 15.1
        },
        "Diluted": {
            "CFY": 18.2,
            "PFY": 14.8
        }
    },
    "Company Information": {
        "Name": "Singapore Ventures Ltd",
        "Financial Year": "2024",
        "Reporting Currency": "SGD"
    }
}

In [8]:
from pydantic_ai import Agent, ModelRetry
# import logfire
# logfire.configure()

model_name = 'google-gla:gemini-1.5-pro' if USE_GEMINI else 'openai:gpt-4o-mini'

# Define the agent
financial_statement_agent = Agent(
    model_name,
    result_type=StatementOfProfitOrLoss,
    system_prompt=FINANCIAL_STATEMENT_PROMPT
)

In [9]:
import json
# Convert the dummy data to a JSON string (optionally, you can use indent=4 for readability)
dummy_data_json = json.dumps(dummy_profit_loss_data, indent=4)

# Use an f-string to inject the dummy data into your prompt:
result = await financial_statement_agent.run(
    f'please map my data here: {dummy_data_json}'
)

In [10]:
print(result.data)

revenue=YearlyValue(CFY=12500000.0, PFY=11250000.0) other_income=YearlyValue(CFY=350000.0, PFY=275000.0) employee_benefits_expense=YearlyValue(CFY=-4750000.0, PFY=-4250000.0) depreciation_expense=YearlyValue(CFY=-780000.0, PFY=-720000.0) amortisation_expense=YearlyValue(CFY=-180000.0, PFY=-165000.0) repairs_maintenance_expense=YearlyValue(CFY=-320000.0, PFY=-280000.0) sales_marketing_expense=YearlyValue(CFY=-1450000.0, PFY=-1350000.0) other_expenses=YearlyValue(CFY=-970000.0, PFY=-850000.0) other_gains_losses=YearlyValue(CFY=85000.0, PFY=-120000.0) finance_costs_net=YearlyValue(CFY=-325000.0, PFY=-290000.0) share_of_profit_loss_associates=YearlyValue(CFY=620000.0, PFY=580000.0) profit_loss_before_taxation=YearlyValue(CFY=4780000.0, PFY=4080000.0) income_tax_expense_benefit=YearlyValue(CFY=-860000.0, PFY=-735000.0) profit_loss_discontinued_operations=YearlyValue(CFY=-150000.0, PFY=-275000.0) total_profit_loss=YearlyValue(CFY=3770000.0, PFY=3070000.0) profit_loss_attributable_to_owners=Y

In [11]:
from devtools import debug
debug(result)

C:\Users\almos\AppData\Local\Temp\ipykernel_11316\702232478.py:2 <module>
    result: AgentRunResult(
        data=StatementOfProfitOrLoss(
            revenue=YearlyValue(
                CFY=12500000.0,
                PFY=11250000.0,
            ),
            other_income=YearlyValue(
                CFY=350000.0,
                PFY=275000.0,
            ),
            employee_benefits_expense=YearlyValue(
                CFY=-4750000.0,
                PFY=-4250000.0,
            ),
            depreciation_expense=YearlyValue(
                CFY=-780000.0,
                PFY=-720000.0,
            ),
            amortisation_expense=YearlyValue(
                CFY=-180000.0,
                PFY=-165000.0,
            ),
            repairs_maintenance_expense=YearlyValue(
                CFY=-320000.0,
                PFY=-280000.0,
            ),
            sales_marketing_expense=YearlyValue(
                CFY=-1450000.0,
                PFY=-1350000.0,
            ),
 

AgentRunResult(data=StatementOfProfitOrLoss(revenue=YearlyValue(CFY=12500000.0, PFY=11250000.0), other_income=YearlyValue(CFY=350000.0, PFY=275000.0), employee_benefits_expense=YearlyValue(CFY=-4750000.0, PFY=-4250000.0), depreciation_expense=YearlyValue(CFY=-780000.0, PFY=-720000.0), amortisation_expense=YearlyValue(CFY=-180000.0, PFY=-165000.0), repairs_maintenance_expense=YearlyValue(CFY=-320000.0, PFY=-280000.0), sales_marketing_expense=YearlyValue(CFY=-1450000.0, PFY=-1350000.0), other_expenses=YearlyValue(CFY=-970000.0, PFY=-850000.0), other_gains_losses=YearlyValue(CFY=85000.0, PFY=-120000.0), finance_costs_net=YearlyValue(CFY=-325000.0, PFY=-290000.0), share_of_profit_loss_associates=YearlyValue(CFY=620000.0, PFY=580000.0), profit_loss_before_taxation=YearlyValue(CFY=4780000.0, PFY=4080000.0), income_tax_expense_benefit=YearlyValue(CFY=-860000.0, PFY=-735000.0), profit_loss_discontinued_operations=YearlyValue(CFY=-150000.0, PFY=-275000.0), total_profit_loss=YearlyValue(CFY=3770